## 初始化信息

In [1]:
import logging
import os
import cv2
import numpy as np
from pathlib import Path
from ultralytics import YOLO

# 加载模型
model = YOLO('yolov8n.pt')  # 使用适合你的YOLO模型
# 禁用所有日志输出
logging.disable(logging.CRITICAL)
# 获取类别名称
class_names = [
    "person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck", "boat",
    "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat", "dog",
    "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
    "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
    "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup", "fork",
    "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli", "carrot", "hot dog",
    "pizza", "donut", "cake", "chair", "couch", "potted plant", "bed", "dining table", "toilet", "tv",
    "laptop", "mouse", "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink", "refrigerator",
    "book", "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush"
]
class_metrics = {class_name: {'TP': 0, 'FP': 0, 'FN': 0} for class_name in class_names}


## IOU计算函数

In [2]:
def calculate_iou(pred_bbox, true_bbox, image_width, image_height):
    # 将预测框坐标从像素转换为相对坐标
    pred_x_center = pred_bbox[0]
    pred_y_center = pred_bbox[1]
    pred_width = pred_bbox[2]
    pred_height = pred_bbox[3]
    
    true_x_center = true_bbox[0] * image_width
    true_y_center = true_bbox[1] * image_height
    true_width = true_bbox[2] * image_width
    true_height = true_bbox[3] * image_height

    # 计算预测框和真实框的四个角的坐标（归一化）
    pred_x1 = pred_x_center - pred_width / 2
    pred_y1 = pred_y_center - pred_height / 2
    pred_x2 = pred_x_center + pred_width / 2
    pred_y2 = pred_y_center + pred_height / 2

    true_x1 = true_x_center - true_width / 2
    true_y1 = true_y_center - true_height / 2
    true_x2 = true_x_center + true_width / 2
    true_y2 = true_y_center + true_height / 2

    # 计算交集
    inter_x1 = max(pred_x1, true_x1)
    inter_y1 = max(pred_y1, true_y1)
    inter_x2 = min(pred_x2, true_x2)
    inter_y2 = min(pred_y2, true_y2)

    inter_area = max(0, inter_x2 - inter_x1) * max(0, inter_y2 - inter_y1)
    pred_area = (pred_x2 - pred_x1) * (pred_y2 - pred_y1)
    true_area = (true_x2 - true_x1) * (true_y2 - true_y1)

    union_area = pred_area + true_area - inter_area
    return inter_area / union_area if union_area > 0 else 0

def compare_results(image_file):
    image = cv2.imread(str(image_file))
    image_height, image_width, _ = image.shape  # 获取图像的宽度和高度

    # 进行YOLO推理
    results = model(image)
    
    # 获取YOLO推理的结果
    preds = results[0].boxes  # 获得框的位置和类别信息
    pred_labels = preds.cls.cpu().numpy().astype(int)
    pred_bboxes = preds.xywh.cpu().numpy()  # 获取相对坐标: [x_center, y_center, width, height]

    # 获取标注文件路径
    label_file = annotations_path / (image_file.stem + '.txt')  # 使用 Path 对象进行路径拼接
    if not label_file.exists():
        return None  # 如果没有对应的标签文件，跳过

    # 读取标注
    true_labels = []
    true_bboxes = []
    with open(label_file, 'r') as f:
        for line in f:
            parts = line.strip().split()
            true_labels.append(int(parts[0]))  # 类别标签
            true_bboxes.append(list(map(float, parts[1:])))  # 边界框位置（归一化的）

    # 计算IOU并判断预测结果是否正确
    total_pred = len(pred_labels)
    total_true = len(true_labels)
    
    for i in range(min(total_pred, total_true)):  # 比较最小数量的预测框和标注框
        pred_label = pred_labels[i]
        pred_bbox = pred_bboxes[i]
        true_label = true_labels[i]
        true_bbox = true_bboxes[i]

        # 检查是否类别匹配，且计算IOU
        if pred_label == true_label:
            iou = calculate_iou(pred_bbox, true_bbox, image_width, image_height)
            if iou > 0.5:  
                # 更新TP
                class_metrics[class_names[pred_label]]['TP'] += 1
            else:
                # 更新FP
                class_metrics[class_names[pred_label]]['FP'] += 1

        else:
            # 如果预测的标签与真实标签不一致，更新FP和FN
            class_metrics[class_names[pred_label]]['FP'] += 1
            class_metrics[class_names[true_label]]['FN'] += 1

    return total_pred, total_true



## 结果打印函数

In [3]:
import pandas as pd
from IPython.display import display

def display_metrics(class_labels, precision, recall, f1_score):
    """
    将 Precision, Recall, F1-Score 数据汇总到 DataFrame 并显示
    
    参数:
    class_labels (list): 类别标签列表
    precision (list): 每个类别的 Precision 列表
    recall (list): 每个类别的 Recall 列表
    f1_score (list): 每个类别的 F1-Score 列表
    """
    # 将数据汇总到 DataFrame
    data = {
        'Class': class_labels,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1_score
    }
    df = pd.DataFrame(data)
    # print(df) #需要复制结果时启用
    # 显示表格
    display(df)

# 调用示例：
# display_metrics(class_labels, precision, recall, f1_score)


## 正常光照测试

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pathlib import Path

# 使用相对路径构建验证集路径和标注路径
val_path = Path(r'.\data\Val\high')  # 使用原始字符串避免转义问题

image_files = list(val_path.glob('*.jpg'))
print(f"Total number of images in the validation set: {len(image_files)}")

# 设置标注文件路径
annotations_path = Path(r'.\data\Annotations (high)\Labels')  # 使用原始字符串
class_metrics = {class_name: {'TP': 0, 'FP': 0, 'FN': 0} for class_name in class_names}
# 对所有图像进行评估
precision = []
recall = []
f1_score = []
class_labels = []

# 对所有图像进行评估
for image_file in image_files:
    compare_results(image_file)

# 计算每个类别的Precision, Recall, F1-Score
# 计算 Precision, Recall, F1-Score
for class_name in class_names:
    TP = class_metrics[class_name]['TP']
    FP = class_metrics[class_name]['FP']
    FN = class_metrics[class_name]['FN']

    Precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    Recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    F1_Score = 2 * (Precision * Recall) / (Precision + Recall) if (Precision + Recall) > 0 else 0
    if Precision + Recall + F1_Score > 0:
        precision.append(Precision)
        recall.append(Recall)
        f1_score.append(F1_Score)
        class_labels.append(class_name)
# 将 Precision, Recall, F1-Score 数据汇总到 DataFrame
data = {
    'Class': class_labels,
    'Precision': precision,
    'Recall': recall,
    'F1-Score': f1_score
}

df = pd.DataFrame(data)

display_metrics(class_labels, precision, recall, f1_score)

Total number of images in the validation set: 3000


,Class,Precision,Recall,F1-Score
0,person,0.987860,0.995413,0.991622
1,bicycle,1.000000,1.000000,1.000000
2,car,0.995805,0.997517,0.996660
3,motorcycle,0.986216,0.992434,0.989315
4,airplane,1.000000,1.000000,1.000000
5,bus,0.989437,0.989437,0.989437
6,train,1.000000,1.000000,1.000000
7,truck,0.997067,0.998532,0.997799
8,boat,1.000000,1.000000,1.000000
9,traffic light,1.000000,1.000000,1.000000


## 低光照测试

In [5]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pathlib import Path

# 使用相对路径构建验证集路径和标注路径
val_path = Path(r'.\data\Val\low')  # 低光照验证集路径
image_files = list(val_path.glob('*.jpg'))
print(f"Total number of images in the validation set: {len(image_files)}")

# 设置标注文件路径
annotations_path_low = Path(r'.\data\Annotations (low)\Labels')

class_metrics = {class_name: {'TP': 0, 'FP': 0, 'FN': 0} for class_name in class_names}
# 对所有图像进行评估
precision = []
recall = []
f1_score = []
class_labels = []

# 对所有图像进行评估
for image_file in image_files:
    compare_results(image_file)

# 计算每个类别的Precision, Recall, F1-Score
# 计算 Precision, Recall, F1-Score
for class_name in class_names:
    TP = class_metrics[class_name]['TP']
    FP = class_metrics[class_name]['FP']
    FN = class_metrics[class_name]['FN']

    Precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    Recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    F1_Score = 2 * (Precision * Recall) / (Precision + Recall) if (Precision + Recall) > 0 else 0
    if Precision + Recall + F1_Score > 0:
        precision.append(Precision)
        recall.append(Recall)
        f1_score.append(F1_Score)
        class_labels.append(class_name)
# 将 Precision, Recall, F1-Score 数据汇总到 DataFrame
data = {
    'Class': class_labels,
    'Precision': precision,
    'Recall': recall,
    'F1-Score': f1_score
}

df = pd.DataFrame(data)

display_metrics(class_labels, precision, recall, f1_score)

Total number of images in the validation set: 3000


,Class,Precision,Recall,F1-Score
0,person,0.304704,0.423450,0.354395
1,bicycle,0.185185,0.240964,0.209424
2,car,0.496583,0.752064,0.598187
3,motorcycle,0.240153,0.303806,0.268255
4,airplane,0.428571,0.500000,0.461538
5,bus,0.400773,0.429558,0.414667
6,train,0.200000,0.384615,0.263158
7,truck,0.399787,0.445762,0.421525
8,traffic light,0.285366,0.294710,0.289963
9,fire hydrant,0.416667,0.238095,0.303030


### 对test数据集进行扰动

In [6]:
import os
import random

# 文件夹路径
input_folder = './data/Test'
output_base_folder = 'output'  # 存储所有处理图像的根文件夹

# 定义不同的模糊程度（不同核大小）
blur_strengths = [(3, 3), (5, 5), (7, 7), (9, 9), (11, 11), (15, 15)]  # 核大小（宽度, 高度）

# 创建根文件夹
if not os.path.exists(output_base_folder):
    os.makedirs(output_base_folder)

# 亮度和对比度调整函数
def adjust_brightness_contrast(image, brightness=0, contrast=0):
    beta = brightness  # 增加亮度的值
    alpha = contrast / 100.0 + 1.0  # 对比度调整值，默认为1.0
    adjusted_image = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
    return adjusted_image

# 添加噪声的函数
def add_noise(image, mean=0, sigma=25):
    """给图像添加噪声"""
    gaussian_noise = np.random.normal(mean, sigma, image.shape).astype('uint8')
    noisy_image = cv2.add(image, gaussian_noise)
    return noisy_image

# 运动模糊的函数
def motion_blur(image, kernel_size=15):
    """添加运动模糊"""
    kernel = np.zeros((kernel_size, kernel_size))
    kernel[int((kernel_size-1)/2), :] = np.ones(kernel_size)
    kernel = kernel / kernel_size
    motion_blurred = cv2.filter2D(image, -1, kernel)
    return motion_blurred

# 遮挡图像的函数
def add_occlusion(image, occlusion_type="rectangle", size=(100, 100), color=(0, 0, 0)):
    """给图像添加遮挡物"""
    h, w, _ = image.shape
    if occlusion_type == "rectangle":
        # 随机选择一个位置，填充为矩形遮挡
        top_left = (random.randint(0, w - size[0]), random.randint(0, h - size[1]))
        bottom_right = (top_left[0] + size[0], top_left[1] + size[1])
        cv2.rectangle(image, top_left, bottom_right, color, -1)
    elif occlusion_type == "circle":
        # 随机选择圆形遮挡
        center = (random.randint(0, w), random.randint(0, h))
        radius = random.randint(30, 50)
        cv2.circle(image, center, radius, color, -1)
    return image

### 处理test数据集并保存到对应的output文件夹

In [7]:
import cv2
import numpy as np

# 读取文件夹中的所有图像
for filename in os.listdir(input_folder):
    if filename.endswith(('.jpg', '.png', '.jpeg')):  # 检查文件扩展名
        image_path = os.path.join(input_folder, filename)
        image = cv2.imread(image_path)

        # 调亮
        brightness_contrast_folder = os.path.join(output_base_folder, 'brighter')
        if not os.path.exists(brightness_contrast_folder):
            os.makedirs(brightness_contrast_folder)
        adjusted_image = adjust_brightness_contrast(image, brightness=50, contrast=30)
        save_path = os.path.join(brightness_contrast_folder, filename)
        cv2.imwrite(save_path, adjusted_image)

        # 调暗
        brightness_contrast_folder = os.path.join(output_base_folder, 'blacker')
        if not os.path.exists(brightness_contrast_folder):
            os.makedirs(brightness_contrast_folder)
        adjusted_image = adjust_brightness_contrast(image, brightness=-50, contrast=0)
        save_path = os.path.join(brightness_contrast_folder, filename)
        cv2.imwrite(save_path, adjusted_image)

        # 应用不同程度的高斯模糊
        for blur_strength in blur_strengths:
            blur_folder = os.path.join(output_base_folder, f"blur_{blur_strength[0]}_{blur_strength[1]}")
            if not os.path.exists(blur_folder):
                os.makedirs(blur_folder)
            blurred_image = cv2.GaussianBlur(image, blur_strength, 0)
            save_path = os.path.join(blur_folder, filename)
            cv2.imwrite(save_path, blurred_image)

        # 添加噪声
        noise_folder = os.path.join(output_base_folder, 'noise')
        if not os.path.exists(noise_folder):
            os.makedirs(noise_folder)
        noisy_image = add_noise(image, sigma=0.1)
        save_path = os.path.join(noise_folder, filename)
        cv2.imwrite(save_path, noisy_image)

        # 添加运动模糊
        motion_blur_folder = os.path.join(output_base_folder, 'motion_blur')
        if not os.path.exists(motion_blur_folder):
            os.makedirs(motion_blur_folder)
        motion_blurred_image = motion_blur(image, 5)
        save_path = os.path.join(motion_blur_folder, filename)
        cv2.imwrite(save_path, motion_blurred_image)

        # 添加遮挡（矩形遮挡）
        occlusion_folder = os.path.join(output_base_folder, 'occlusion_rectangle')
        if not os.path.exists(occlusion_folder):
            os.makedirs(occlusion_folder)
        image_with_occlusion = add_occlusion(image.copy(), occlusion_type="rectangle", size=(100, 100), color=(0, 0, 0))
        save_path = os.path.join(occlusion_folder, filename)
        cv2.imwrite(save_path, image_with_occlusion)

        # 添加圆形遮挡
        occlusion_circle_folder = os.path.join(output_base_folder, 'occlusion_circle')
        if not os.path.exists(occlusion_circle_folder):
            os.makedirs(occlusion_circle_folder)
        image_with_circle_occlusion = add_occlusion(image.copy(), occlusion_type="circle", size=(0, 0),
                                                    color=(0, 0, 0))
        save_path = os.path.join(occlusion_circle_folder, filename)
        cv2.imwrite(save_path, image_with_circle_occlusion)

        # 压缩处理
        jpeg_compressed_folder = os.path.join(output_base_folder, 'jpeg_compressed')
        if not os.path.exists(jpeg_compressed_folder):
            os.makedirs(jpeg_compressed_folder)
        compression_params = [cv2.IMWRITE_JPEG_QUALITY, 50]
        # 保存压缩后的图像
        save_path = os.path.join(jpeg_compressed_folder, filename)
        cv2.imwrite(save_path, image, compression_params)

        print(f"Processed {filename}")

print("All images processed and saved to respective folders.")


Processed Test_1.jpg
Processed Test_10.jpg
Processed Test_100.jpg
Processed Test_1000.jpg
Processed Test_101.jpg
Processed Test_102.jpg
Processed Test_103.jpg
Processed Test_104.jpg
Processed Test_105.jpg
Processed Test_106.jpg
Processed Test_107.jpg
Processed Test_108.jpg
Processed Test_109.jpg
Processed Test_11.jpg
Processed Test_110.jpg
Processed Test_111.jpg
Processed Test_112.jpg
Processed Test_113.jpg
Processed Test_114.jpg
Processed Test_115.jpg
Processed Test_116.jpg
Processed Test_117.jpg
Processed Test_118.jpg
Processed Test_119.jpg
Processed Test_12.jpg
Processed Test_120.jpg
Processed Test_121.jpg
Processed Test_122.jpg
Processed Test_123.jpg
Processed Test_124.jpg
Processed Test_125.jpg
Processed Test_126.jpg
Processed Test_127.jpg
Processed Test_128.jpg
Processed Test_129.jpg
Processed Test_13.jpg
Processed Test_130.jpg
Processed Test_131.jpg
Processed Test_132.jpg
Processed Test_133.jpg
Processed Test_134.jpg
Processed Test_135.jpg
Processed Test_136.jpg
Processed Test_1

### 推理函数，并计算IOU

In [8]:
def inference(image_file, model):
    image = cv2.imread(str(image_file))
    image_height, image_width, _ = image.shape  # 获取图像的宽度和高度

    results = model(image)

    # 获取YOLO推理的结果
    preds = results[0].boxes  # 获得框的位置和类别信息
    pred_labels = preds.cls.cpu().numpy().astype(int)
    pred_bboxes = preds.xywh.cpu().numpy()  # 获取相对坐标: [x_center, y_center, width, height]

    return pred_labels, pred_bboxes, image_height, image_width

def calculate_iou_test(pred_bbox, true_bbox):
    # 将预测框坐标从像素转换为相对坐标
    pred_x_center = pred_bbox[0]
    pred_y_center = pred_bbox[1]
    pred_width = pred_bbox[2]
    pred_height = pred_bbox[3]

    true_x_center = true_bbox[0]
    true_y_center = true_bbox[1]
    true_width = true_bbox[2]
    true_height = true_bbox[3]

    # 计算预测框和真实框的四个角的坐标（归一化）
    pred_x1 = pred_x_center - pred_width / 2
    pred_y1 = pred_y_center - pred_height / 2
    pred_x2 = pred_x_center + pred_width / 2
    pred_y2 = pred_y_center + pred_height / 2

    true_x1 = true_x_center - true_width / 2
    true_y1 = true_y_center - true_height / 2
    true_x2 = true_x_center + true_width / 2
    true_y2 = true_y_center + true_height / 2

    # 计算交集
    inter_x1 = max(pred_x1, true_x1)
    inter_y1 = max(pred_y1, true_y1)
    inter_x2 = min(pred_x2, true_x2)
    inter_y2 = min(pred_y2, true_y2)

    inter_area = max(0, inter_x2 - inter_x1) * max(0, inter_y2 - inter_y1)
    pred_area = (pred_x2 - pred_x1) * (pred_y2 - pred_y1)
    true_area = (true_x2 - true_x1) * (true_y2 - true_y1)

    union_area = pred_area + true_area - inter_area
    return inter_area / union_area if union_area > 0 else 0

def cal(pred, org):
    pred_labels, pred_bboxes, image_height, image_width = pred
    true_labels, true_bboxes,_, _ = org
    # 计算IOU并判断预测结果是否正确
    correct = 0
    total_pred = len(pred_labels)
    total_true = len(true_labels)

    for i in range(min(total_pred, total_true)):  # 比较最小数量的预测框和标注框
        pred_label = pred_labels[i]
        pred_bbox = pred_bboxes[i]
        true_label = true_labels[i]
        true_bbox = true_bboxes[i]

        # 检查是否类别匹配，且计算IOU
        if pred_label == true_label:
            iou = calculate_iou_test(pred_bbox, true_bbox)
            # print(f'Predicted IoU: {iou:.4f}, Predicted: {pred_bbox}, True: {true_bbox}')
            if iou > 0.5:
                correct += 1

    return correct / total_pred if total_pred > 0 else 0  # 返回正确率

### 执行推理

In [9]:
import csv


image_files = list(Path(input_folder).glob('*.jpg'))
print(f"Total number of images in the validation set: {len(image_files)}")
# 设置标注文件路径
acc = {}
accuracies_ori = []
for image_file in image_files:
    origin_result = inference(image_file, model)
    origin_result2 = inference(image_file, model)
    accuracy2 = cal(origin_result, origin_result2)
    if accuracy2 is not None:
        accuracies_ori.append(accuracy2)
std_mean_accuracy = np.mean(accuracies_ori)

acc['std'] = std_mean_accuracy
print(std_mean_accuracy)

ops = ['blur_3_3', 'blur_5_5', 'blur_7_7', 'blur_9_9', 'blur_11_11', 'blur_15_15', 'brighter', 'motion_blur', 'blacker', 'noise', 'occlusion_circle', 'occlusion_rectangle', 'jpeg_compressed']
ops = ['noise']
for op in ops:
    acc_op = []
    for image_file in image_files:
        filepath = output_base_folder + '/' + op + '/' + image_file.parts[-1]
        result = inference(filepath, model)
        origin_result = inference(image_file, model)
        acc_op.append(cal(result, origin_result))
    acc[op] = np.mean(acc_op)
    print(acc[op])
print(acc)

Total number of images in the validation set: 1000
0.987
0.7704007406622857
{'std': 0.987, 'noise': 0.7704007406622857}


### 保存数据

In [10]:
with open('res.csv', mode='w', newline='') as f:
    writer = csv.writer(f)
    # 写入表头
    writer.writerow(acc.keys())
    # 写入数据
    writer.writerow(acc.values())